In [7]:
import pandas as pd

posts = pd.read_csv("ptt_stock_posts.csv", index_col=0)
import json
push_df = pd.DataFrame(json.loads(posts.Pushes[0])).T

,Title,Author,Date,Content,Link,Pushes
0,[閒聊] 2025/03/10 盤中閒聊,laptic (靜夜聖林彼岸花),Mon Mar 10 08:30:01 2025,\n==================114/03/10台股資訊重點整理，供股民做投資參考...,https://www.ptt.cc/bbs/Stock/M.1741566604.A.34...,"{""0"":{""Tag"":""PUSH"",""Userid"":""EvilJustice"",""Con..."


In [21]:
import json
push_df = pd.DataFrame(json.loads(posts.Pushes[0])).T

In [1]:
import openai
openai.__version__

'1.65.5'

In [25]:
!pip install python_dotenv

  Obtaining dependency information for python_dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [41]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class SentimentAgent:
    
    def __init__(self, systemPrompt=""):
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        self.system = systemPrompt
        self.messages = []
        if self.system:
            self.messages.append({"role": "system",
                                  "content": [{"type": "text", "text": self.system}]})
            

    def __call__(self, prompt):
        self.messages.append({"role": "user",
                              "content": [{"type": "text", "text": prompt}]})
        result = self.get_completions()
        self.messages.append({"role": "assistant",
                              "content": [{"type": "text", "text": result}]})
        return result
    
    def get_completions(self, model="gpt-4o"):
        response = client.chat.completions.create(
            model=model,
            messages=self.messages,
            temperature=0
        )

        return response.choices[0].message.content



In [42]:
systemy_prompt = """對金融文本進行情緒分析。

分析給定的金融文本並識別其情緒，包括正面、負面或中性情感。確保先進行合理的分析和推理，然後得出情緒結論。

# 步驟

1. **文本理解**：仔細閱讀並理解給定的金融文本。
2. **語境分析**：識別可能影響情緒的上下文或特定語言。
3. **情感識別**：根據詞彙、語氣和語境識別情感特徵。
4. **推理**：綜合所有識別的信息形成初步判斷。
5. **情緒結論**：明確文本的情緒為正面、負面或中性。
6.標的識別：識別是否針對某一個標的，或是大盤指數。如果毫無關係，請回答"無"
# 輸出格式

以簡單的文本格式表述情感結論。例如:
- "正面"
- "負面"
- "中性"
同時附上標的。

# 示例

- **輸入**: "台積電這家公司的季度利潤超出了市場預期，投資者反應積極。"
  **推理**: 公司利潤超出預期通常是利好消息，市場反應積極進一步增強了這一點。
  **輸出**: "正面，台積電" 

- **輸入**: "Google 宣布了一項裁員計劃，引發員工恐慌並導致股價下跌。"
  **推理**: 裁員計劃和股價下跌通常被視為負面消息，並可能引發市場和內部的負面情緒。
  **輸出**: "負面，Google"

# 注意事項

- 確保在做出情緒判斷前，進行充分的分析和推理。
- 考慮到金融文本中的術語和特有表達，注意其可能產生的情感影響。"""

In [43]:
agent = SentimentAgent(systemy_prompt)

user_prompt = "文本：馬斯克帶衰！ 特斯拉創史上最慘跌勢 專家急喊不妙了"
agent(user_prompt)

'**推理**: 文本中使用了「帶衰」、「最慘跌勢」和「不妙」等詞語，這些詞語都帶有強烈的負面情緒，表明特斯拉的股價或市場表現不佳，並且專家對此表示擔憂。\n\n**輸出**: "負面，特斯拉"'

In [44]:
agent("文本：美國總統川普決定增加加拿大的 AI 晶片關稅")

'**推理**: 文本中提到美國總統決定增加加拿大的AI晶片關稅。增加關稅通常會被視為貿易壁壘，可能對相關行業或公司造成負面影響，特別是對加拿大的AI晶片出口商來說。然而，這一政策的影響可能是中性的，因為它取決於具體的市場反應和雙方的應對措施。\n\n**輸出**: "中性，無"'

In [45]:
agent.messages

[{'role': 'system',
  'content': [{'type': 'text',
    'text': '對金融文本進行情緒分析。\n\n分析給定的金融文本並識別其情緒，包括正面、負面或中性情感。確保先進行合理的分析和推理，然後得出情緒結論。\n\n# 步驟\n\n1. **文本理解**：仔細閱讀並理解給定的金融文本。\n2. **語境分析**：識別可能影響情緒的上下文或特定語言。\n3. **情感識別**：根據詞彙、語氣和語境識別情感特徵。\n4. **推理**：綜合所有識別的信息形成初步判斷。\n5. **情緒結論**：明確文本的情緒為正面、負面或中性。\n6.標的識別：識別是否針對某一個標的，或是大盤指數。如果毫無關係，請回答"無"\n# 輸出格式\n\n以簡單的文本格式表述情感結論。例如:\n- "正面"\n- "負面"\n- "中性"\n同時附上標的。\n\n# 示例\n\n- **輸入**: "台積電這家公司的季度利潤超出了市場預期，投資者反應積極。"\n  **推理**: 公司利潤超出預期通常是利好消息，市場反應積極進一步增強了這一點。\n  **輸出**: "正面，台積電" \n\n- **輸入**: "Google 宣布了一項裁員計劃，引發員工恐慌並導致股價下跌。"\n  **推理**: 裁員計劃和股價下跌通常被視為負面消息，並可能引發市場和內部的負面情緒。\n  **輸出**: "負面，Google"\n\n# 注意事項\n\n- 確保在做出情緒判斷前，進行充分的分析和推理。\n- 考慮到金融文本中的術語和特有表達，注意其可能產生的情感影響。'}]},
 {'role': 'user',
  'content': [{'type': 'text', 'text': '文本：馬斯克帶衰！ 特斯拉創史上最慘跌勢 專家急喊不妙了'}]},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': '**推理**: 文本中使用了「帶衰」、「最慘跌勢」和「不妙」等詞語，這些詞語都帶有強烈的負面情緒，表明特斯拉的股價或市場表現不佳，並且專家對此表示擔憂。\n\n**輸出**: "負面，特斯拉"'}]},
 {'role': 'user',
  'content':

In [46]:
agent("我剛剛問你的情緒分析結果，分別是什麼？")

'1. **特斯拉文本**: "負面，特斯拉"\n2. **AI晶片關稅文本**: "中性，無"'

In [64]:
def sentiment_prompt_all(stock_market_chat):
    return f"""
請分析以下一系列的股票市場聊天內容情緒。
每則聊天內容都有對應的編號，整個聊天內容的集合以三個反引號（```）括住。
判斷每則聊天內容的情緒是"positive" or "negative" or "neutral"。
您的答案必須是單一詞彙 "positive" or "negative" or "neutral。

如果聊天內容中沒有提及特定的股票目標，請將情緒標記為 "neutral"。
如果提到了股票目標，請將其列出，使用中文名稱、股票代碼或代號。
如果沒有提及股票代碼或名稱，請將該欄位設為空字串 ""。

請依照以下 JSON 格式提供答案，使其能夠直接轉換為 Pandas DataFrame，並以尖括號（<>）分隔的鍵值對應每則聊天內容：
- sentiment（情緒）：您的答案必須是 "positive"、"negative" 或 "neutral"。如果聊天內容未提及股票目標，請標記為 "neutral"。
- stocktarget（標的）：請列出相關的股票名稱、股票代碼或代號。如果未提及，請將其設為空字串 ""。

請使用以下格式回傳結果：
{{
    "the corresponding number" :{{"sentiment":..., "stocktarget":...}},
    "the corresponding number" :{{"sentiment":..., "stocktarget":...}},
    "the corresponding number" :{{"sentiment":..., "stocktarget":...}},
    ...
}}

其中 "the corresponding number" 必須與 "Stock_market_chat"（股票市場聊天內容）中的 index 相對應。

Stock_market_chat text: ```{stock_market_chat}
請確保嚴格遵守上述 JSON 格式和規則，並確保順序正確，避免回傳冗餘資料。
"""

In [48]:
print(push_df)

        Tag        Userid                          Content    Ip   Ipdatetime
0      PUSH   EvilJustice                      等藍燈抄底！！！！！！  None  03/10 08:30
1       BOO      King5566                               媽媽  None  03/10 08:30
2      PUSH    firemothra   http://i.imgur.com/1L4ZhxW.jpg  None  03/10 08:30
3      PUSH      lf3netxx  https://i.imgur.com/5FbjSEi.png  None  03/10 08:30
4      PUSH         NKUHT                      今天噴鋼鐵水泥卡打掐？  None  03/10 08:30
...     ...           ...                              ...   ...          ...
4783    BOO      a0808996                      腦大 博智賺到流湯!!  None  03/10 14:07
4784   PUSH  fantasyscure           八大官股行庫國安基金和市場對作 沒肉吃 看戲  None  03/10 14:11
4785  ARROW   anthony1990                              VV叫  None  03/10 14:40
4786   PUSH    ice2240580       6天前叫朋友把台股賣光 朋友賣是賣了 但因此吵架 現  None  03/10 14:59
4787  ARROW    ice2240580                    在想想應該不要多管閒事==  None  03/10 15:00

[4788 rows x 5 columns]


In [53]:
len(push_df)

4788

In [58]:
push_filtered = push_df[~push_df.Content.str.contains("http")]

In [60]:
push_filtered.reset_index(inplace=True)

In [61]:
push_filtered

,index,Tag,Userid,Content,Ip,Ipdatetime
0,0,PUSH,EvilJustice,等藍燈抄底！！！！！！,None,03/10 08:30
1,1,BOO,King5566,媽媽,None,03/10 08:30
2,4,PUSH,NKUHT,今天噴鋼鐵水泥卡打掐？,None,03/10 08:30
3,5,PUSH,lepidoptera,今天屎定了,None,03/10 08:30
4,6,PUSH,abc5555990,每周一必撿鑽石,None,03/10 08:30
...,...,...,...,...,...,...
4564,4783,BOO,a0808996,腦大 博智賺到流湯!!,None,03/10 14:07
4565,4784,PUSH,fantasyscure,八大官股行庫國安基金和市場對作 沒肉吃 看戲,None,03/10 14:11
4566,4785,ARROW,anthony1990,VV叫,None,03/10 14:40
4567,4786,PUSH,ice2240580,6天前叫朋友把台股賣光 朋友賣是賣了 但因此吵架 現,None,03/10 14:59


In [65]:
i = 0
sentiment_prompt_all(push_filtered[i:i+50]["Content"])

'\n請分析以下一系列的股票市場聊天內容情緒。\n每則聊天內容都有對應的編號，整個聊天內容的集合以三個反引號（```）括住。\n判斷每則聊天內容的情緒是"positive" or "negative" or "neutral"。\n您的答案必須是單一詞彙 "positive" or "negative" or "neutral。\n\n如果聊天內容中沒有提及特定的股票目標，請將情緒標記為 "neutral"。\n如果提到了股票目標，請將其列出，使用中文名稱、股票代碼或代號。\n如果沒有提及股票代碼或名稱，請將該欄位設為空字串 ""。\n\n請依照以下 JSON 格式提供答案，使其能夠直接轉換為 Pandas DataFrame，並以尖括號（<>）分隔的鍵值對應每則聊天內容：\n- sentiment（情緒）：您的答案必須是 "positive"、"negative" 或 "neutral"。如果聊天內容未提及股票目標，請標記為 "neutral"。\n- stocktarget（標的）：請列出相關的股票名稱、股票代碼或代號。如果未提及，請將其設為空字串 ""。\n\n請使用以下格式回傳結果：\n{\n    "the corresponding number" :{"sentiment":..., "stocktarget":...},\n    "the corresponding number" :{"sentiment":..., "stocktarget":...},\n    "the corresponding number" :{"sentiment":..., "stocktarget":...},\n    ...\n}\n\n其中 "the corresponding number" 必須與 "Stock_market_chat"（股票市場聊天內容）中的 index 相對應。\n\nStock_market_chat text: ```0                等藍燈抄底！！！！！！\n1                         媽媽\n2                今天噴鋼鐵水泥卡打掐？\n3                      今天屎定了\n4                    每周一必撿鑽石\n5       

In [68]:
def sentiment_system_prompt():
    return f"""
請分析以下一系列的股票市場聊天內容情緒。
每則聊天內容都有對應的編號，整個聊天內容的集合以三個反引號（```）括住。
判斷每則聊天內容的情緒是"positive" or "negative" or "neutral"。
您的答案必須是單一詞彙 "positive" or "negative" or "neutral。

如果聊天內容中沒有提及特定的股票目標，請將情緒標記為 "neutral"。
如果提到了股票目標，請將其列出，使用中文名稱、股票代碼或代號。
如果沒有提及股票代碼或名稱，請將該欄位設為空字串 ""。

請依照以下 JSON 格式提供答案，使其能夠直接轉換為 Pandas DataFrame，並以尖括號（<>）分隔的鍵值對應每則聊天內容：
- sentiment（情緒）：您的答案必須是 "positive"、"negative" 或 "neutral"。如果聊天內容未提及股票目標，請標記為 "neutral"。
- stocktarget（標的）：請列出相關的股票名稱、股票代碼或代號。如果未提及，請將其設為空字串 ""。

請使用以下格式回傳結果：
{{
    "the corresponding number" :{{"sentiment":..., "stocktarget":...}},
    "the corresponding number" :{{"sentiment":..., "stocktarget":...}},
    "the corresponding number" :{{"sentiment":..., "stocktarget":...}},
    ...
}}

其中 "the corresponding number" 必須與 "Stock_market_chat"（股票市場聊天內容）中的 index 相對應。

請確保嚴格遵守上述 JSON 格式和規則，並確保順序正確，避免回傳冗餘資料。
"""

In [67]:
def sentiment_user_prompt(stock_market_chat):
    return f"Stock_market_chat text: ```{stock_market_chat}```"


In [69]:
agent = SentimentAgent(sentiment_system_prompt())

In [79]:
i = 0
delta = 50
all_results = []
# while i < len(push_filtered) + delta:
while i < 252 - delta:
    agent = SentimentAgent(sentiment_system_prompt())
    response = agent(sentiment_user_prompt(push_filtered[i:i+delta]["Content"]))
    result_df = pd.DataFrame(json.loads(response.strip("```").lstrip("json"))).T
    print(result_df)
    all_results.append(result_df)
    i += delta

final_df = pd.concat(all_results, ignore_index=True)


   sentiment stocktarget
0    neutral            
1    neutral            
2    neutral            
3   negative            
4   positive            
5    neutral            
6   negative            
7   negative            
8    neutral            
9   negative            
10   neutral            
11   neutral            
12   neutral            
13  negative            
14  negative         比特幣
15   neutral            
16   neutral            
17   neutral            
18   neutral            
19   neutral            
20  negative            
21  negative            
22   neutral            
23  positive            
24   neutral      1000GG
25  positive         TLT
26  negative            
27   neutral            
28  negative            
29   neutral            
30   neutral            
31  negative            
32   neutral            
33  positive            
34  positive            
35   neutral            
36   neutral            
37   neutral          萬海
38   neutral            


In [80]:
final_df

,sentiment,stocktarget
0,neutral,
1,neutral,
2,neutral,
3,negative,
4,positive,
...,...,...
245,neutral,
246,positive,
247,neutral,
248,neutral,GG


In [84]:
pos = (final_df.sentiment == "positive").sum()

In [85]:
neg = (final_df.sentiment == "negative").sum()

In [88]:
result = (pos - neg) / len(final_df)

In [89]:
result

-0.14

In [94]:
posts = pd.read_csv("2023_07_25.csv", index_col=0)
push_df = pd.DataFrame(json.loads(posts.Pushes[0])).T
push_df

,Tag,Userid,Content,Ip,Ipdatetime,Sentiment,Stocktarget
0,PUSH,TeddyPig5566,肉鬆怎麼酥~~~,None,07/25 08:30,positive,
1,ARROW,zzzzzzzzzzzy,開盤比手速,None,07/25 08:30,neutral,
2,PUSH,Hsiencheng,今天強嘎！,None,07/25 08:30,positive,
3,PUSH,dragonmester,99緯創,None,07/25 08:30,neutral,99緯創
4,PUSH,ipath19,文雯早 大家早,None,07/25 08:30,neutral,
...,...,...,...,...,...,...,...
5294,ARROW,dosiris,怎麼跟我一樣180接 笑鼠,None,07/25 14:02,neutral,
5295,PUSH,luchiya123,https://i.imgur.com/6Cp99ue.jpg,None,07/25 14:02,neutral,
5296,PUSH,dosiris,180-18股息-70股息 今年也解套了,None,07/25 14:04,positive,
5297,PUSH,SimleToMe,樓上不算減資後的股價嗎,None,07/25 14:09,neutral,


In [97]:
pos = (push_df.Sentiment == "positive").sum()
neg = (push_df.Sentiment == "negative").sum()
result = (pos - neg) / len(push_df)
result

0.0007548594074353652

In [100]:
posts = pd.read_csv("2023_07_24.csv", index_col=0)
push_df = pd.DataFrame(json.loads(posts.Pushes[0])).T
pos = (push_df.Sentiment == "positive").sum()
neg = (push_df.Sentiment == "negative").sum()
result0724 = (pos - neg) / len(push_df)
result0724

-0.04902739182215165